In [18]:
#필요한 라이브러리
from zumi.zumi import Zumi
from zumi.util.screen import Screen
from zumi.protocol import Note
from zumi.util.camera import Camera
from zumi.util.color_classifier import ColorClassifier
from threading import Thread
from zumi.util.vision import Vision
import IPython.display
import PIL.Image
import time

zumi = Zumi()
screen = Screen()
camera = Camera()
vision = Vision()
right_speed = 7
left_speed = 15

Starting Zumi 
Pi Zero I2C is available
Verified Pi Zero is the same
Gyroscope previously calibrated
Zumi board detected
OLED Screen detected
Gyroscope & Accelerometer detected
Zumi battery level  60 %


In [ ]:
#검정색 라인을 따라가다가 흰색 라인을 만나면 멈추는 함수
def touch_white():
    try:
        a = time.time()
        while True:
            bottom_right = zumi.read_IR('bottom_right')
            bottom_left = zumi.read_IR('bottom_left')
            front_right = zumi.read_IR('front_right')
            front_left = zumi.read_IR('front_left')

            
            if bottom_right < 100 and bottom_left < 100 :
                print(front_right, front_left)
                zumi.stop()
                break
                        
                   
            
            elif bottom_right > 100 and bottom_left > 100 :
                zumi.control_motors(right_speed, left_speed ,0)

            elif bottom_right > 100 and bottom_left < 100 :
                zumi.control_motors(0,1,0)

            elif bottom_right < 100 and bottom_left > 100 :
                zumi.control_motors(1,0,0)

    except KeyboardInterrupt:
        screen.draw_text_center('Linetrace Stop')
        zumi.stop(0)
        
def go():
    try:
        while True:
            bottom_right = zumi.read_IR('bottom_right')
            bottom_left = zumi.read_IR('bottom_left')
            front_right = zumi.read_IR('front_right')
            front_left = zumi.read_IR('front_left')

            
            if bottom_right < 100 and bottom_left < 100 :
                if(front_right > 50 or front_left > 80):
                    zumi.reverse(1, 0.2)
                else:
                    print(front_right, front_left)
                    zumi.stop()
                    break
            
            elif bottom_right > 100 and bottom_left > 100 :
                zumi.control_motors(right_speed, left_speed ,0)

            elif bottom_right > 100 :
                zumi.control_motors(0,1,0)

            elif bottom_left > 100 :
                zumi.control_motors(1,0,0)

    except KeyboardInterrupt:
        screen.draw_text_center('Linetrace Stop')
        zumi.stop(0)
        
def line():
    touch_white()
    zumi.turn_left()
    touch_white()
    zumi.turn_right()
    touch_white()
    zumi.turn_right()
    touch_white()
        
#주차함수
def parking():
    block()
    time.sleep(2)
    zumi.turn_right()
    zumi.turn_right()
    block()
    
def counting_line(move_time = 2, speed = 20, ir_threshold = 100):

    count = 0
    start_time = time.time()   

    while time.time() - start_time < move_time: 
        zumi.drive_over_markers(1, speed,ir_threshold,5)
        count= count + 1

    zumi.stop()
    zumi.turn_right(7)
    
    return count

def turn_r():
    try:
        while True:
            front_right = zumi.read_IR('front_right')
            front_left = zumi.read_IR('front_left')
            if front_left < 80 or front_right < 50:
                print("something ahead")
                zumi.stop()
                time.sleep(0.5)
                zumi.turn_right()
                break
            else:
                print("nothing detected")
                zumi.control_motors(right_speed, left_speed ,0)

    except KeyboardInterrupt:
        zumi.stop()
    
def turn_l():
    try:
        while True:
            front_right = zumi.read_IR('front_right')
            front_left = zumi.read_IR('front_left')
            if front_left < 80 or front_right < 50:
                print("something ahead")
                zumi.stop()
                time.sleep(0.5)
                zumi.turn_left()
                break
            else:
                print("nothing detected")
                zumi.control_motors(right_speed, left_speed ,0)

    except KeyboardInterrupt:
        zumi.stop()
        
def color_restart():
    
    try:
        user_name = 'babo' # 내 계정 이름
        demo_name = 'please' # 색 인식 저장 파일 이름 (잘 기억해놓기)

        knn = ColorClassifier(user_name=user_name)
        train = knn.load_model(demo_name)
        knn.fit("hsv")

        while True:
            front_right = zumi.read_IR('front_right')
            front_left = zumi.read_IR('front_left')
            camera.start_camera()

            image = camera.capture()
            predict = knn.predict(image)

            if (predict == "red"):
                screen.draw_text_center("red")
            elif (predict == "green"):
                screen.draw_text_center("green")
            elif (predict == "yellow") :
                screen.draw_text_center("yellow")
            else:
                camera.close()
                turn_r()
                turn_r()
                turn_l()
                turn_l()
                break

    except KeyboardInterrupt:
        zumi.stop()


def block():
    while True:
        front_right = zumi.read_IR('front_right')
        front_left = zumi.read_IR('front_left')
        if front_left < 80 or front_right < 50:
            zumi.stop()
            break
        else:
            zumi.control_motors(right_speed, left_speed ,0)
            
def finish():
    while True:
        camera.start_camera()
        image = camera.capture()

        qr_code = vision.find_QR_code(image)
        qr_message = vision.get_QR_message(qr_code) #이게 qr로 인식한 글씨
        
        if(qr_message == None):
            continue
        else:
            print(qr_message)
            screen.draw_text_center(qr_message)
            break

    camera.close()

    if(qr_message == "right"):
        print(qr_message)
        screen.draw_text_center(qr_message)
        zumi.turn_right()
        touch_white()
        zumi.turn_left()
        break
    elif(qr_message == "left"):
        print(qr_message)
        screen.draw_text_center(qr_message)
        zumi.turn_left()
        touch_white()
        zumi.turn_right()
        break

    camera.close()
    touch_white()
    zumi.forward(10, 1.5)

In [40]:
#A course#

while True:
    camera.start_camera()
    image = camera.capture()

    qr_code = vision.find_QR_code(image)
    qr_message = vision.get_QR_message(qr_code) #이게 qr로 인식한 글씨

    if(qr_message == None):
        continue
    else:
        print(qr_message)
        screen.draw_text_center(qr_message)
        break

camera.close()

if(qr_message == "factory"):
    counting_line()
    zumi.turn_right()
elif(qr_message == "school"):
    counting_line()
    counting_line()
    zumi.turn_left(85)
elif(qr_message == "building"):
    counting_line()
    counting_line()
    counting_line()
    zumi.turn_right(78)
else:
    counting_line()
    counting_line()
    counting_line()
    counting_line()
    zumi.turn_left(100)
parking()
if(qr_message == "factory" or qr_message == "building"):
    zumi.turn_right()
else:
    zumi.turn_left()
    
block()
    
zumi.turn_right()
touch_white()

zumi.forward(10, 0, 4)

# B course #
time.sleep(1)
color_restart()
touch_white()

# C course #
zumi.forward(10, 0.4)
line()
finish()

Starting PiCamera
129 156
Starting PiCamera
left
164 187
Closing PiCamera
164 180
